In [1]:
# dependencies
import requests
import json
import pandas as pd
import time
import os

# import spotipy
#!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
from config import spotify_client_ID as sp_client, spotify_client_secret as sp_secret

In [3]:
# set up client credentials 
# https://spotipy.readthedocs.io/en/master/?highlight=spotifyclientcredentials#client-credentials-flow
# API info: https://developer.spotify.com/documentation/web-api/reference/#/

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(sp_client, sp_secret))

In [4]:
# API info: https://developer.spotify.com/documentation/web-api/
## https://developer.spotify.com/documentation/web-api/reference/#/

sp_root_url = 'https://api.spotify.com/v1/'

In [5]:
# load csv
file = "00A_data/top40_1997_2022_raw.csv"

In [6]:
# read csv
top_df = pd.read_csv(file)
top_df.head()

,week,position,song,artist
0,1997-07-07,1,Mmm Bop,Hanson
1,1997-07-07,2,Bitch,Meredith Brooks
2,1997-07-07,3,Say You'll Be There,Spice Girls
3,1997-07-07,4,Return Of The Mack,Mark Morrison
4,1997-07-07,5,Sunny Came Home,Shawn Colvin


In [7]:
# get track id by track name and artist name
top_df_song = top_df['song'].str.replace(' ', '%20')
top_df_song = pd.DataFrame(top_df_song)

In [8]:
top_df_song.head()

,song
0,Mmm%20Bop
1,Bitch
2,Say%20You'll%20Be%20There
3,Return%20Of%20The%20Mack
4,Sunny%20Came%20Home


In [9]:
top_df_artist = top_df['artist'].str.replace(' ', '%20')
top_df_artist = pd.DataFrame(top_df_artist)
top_df_artist.head()

,artist
0,Hanson
1,Meredith%20Brooks
2,Spice%20Girls
3,Mark%20Morrison
4,Shawn%20Colvin


In [10]:
top_dfs = pd.concat([top_df, top_df_song, top_df_artist], axis="columns")
top_dfs

,week,position,song,artist,song,artist
0,1997-07-07,1,Mmm Bop,Hanson,Mmm%20Bop,Hanson
1,1997-07-07,2,Bitch,Meredith Brooks,Bitch,Meredith%20Brooks
2,1997-07-07,3,Say You'll Be There,Spice Girls,Say%20You'll%20Be%20There,Spice%20Girls
3,1997-07-07,4,Return Of The Mack,Mark Morrison,Return%20Of%20The%20Mack,Mark%20Morrison
4,1997-07-07,5,Sunny Came Home,Shawn Colvin,Sunny%20Came%20Home,Shawn%20Colvin
...,...,...,...,...,...,...
52763,2022-10-15,36,Alone,Rod Wave,Alone,Rod%20Wave
52764,2022-10-15,37,Big Time,"DJ Khaled, Future &amp; Lil Baby",Big%20Time,"DJ%20Khaled,%20Future%20&amp;%20Lil%20Baby"
52765,2022-10-15,38,Under The Influence,Chris Brown,Under%20The%20Influence,Chris%20Brown
52766,2022-10-15,39,Romantic Homicide,d4vd,Romantic%20Homicide,d4vd


In [11]:
q_search = []
for a, song in enumerate(top_df_song)

SyntaxError: invalid syntax (998648406.py, line 2)